In [1]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora
from tqdm import tqdm_notebook
from pprint import pprint

# 사용자간의 유사성을 평가하기 위한 measure를 사용하기 위해 불러오기
from gensim.matutils import hellinger
from gensim import matutils

import pandas as pd
import os

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
NUM_TOPICS = 4

if not os.path.exists("kakao(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary.id2token, \
                author2doc=author2doc, passes=10)
    model.save('kakao(ATM)_model')
else:
    model = AuthorTopicModel.load("kakao(ATM)_model")

In [3]:
# 토픽별 분포 확인하기
model.show_topic(1, topn=20)

[('난', 0.012260236931585794),
 ('아', 0.011676997276547715),
 ('락원이', 0.010431516849827774),
 ('동현이', 0.005928134762841963),
 ('오늘', 0.005294233510440971),
 ('다', 0.005287299506846207),
 ('내가', 0.005243276235309308),
 ('근데', 0.004646346629371917),
 ('나', 0.004085344852820157),
 ('아니', 0.004019273777094868),
 ('진짜', 0.004006107460260246),
 ('더', 0.004005512804978402),
 ('내일', 0.0034142604828852276),
 ('그럼', 0.003364904612905689),
 ('이걸', 0.003364637952624334),
 ('니', 0.002730084928202692),
 ('좀', 0.002724732961971116),
 ('락', 0.002723984236604207),
 ('너가', 0.0027237221128403382),
 ('이거', 0.0027237196641392526)]

In [4]:
# 토픽 라벨 지정.
topic_labels = ["Topic0", "Topic1", "Topic2", "Topic3"]

In [5]:
# 토픽별로 topN 단어 확인하기.
for topics in model.show_topics(model.num_topics):
    print("Label : " + topic_labels[topics[0]])
    words = ""
    for word, prob in model.show_topic(topics[0], topn=20):
        words += word + " "
    print("Words :", words)
    print()


Label : Topic0
Words : 왜 형 오늘 락원이 또 진짜 그럼 같이 락원아 새도 태환이형 아 좀 나 니 벡해 배드민턴 걍 그냥 혹시 

Label : Topic1
Words : 난 아 락원이 동현이 오늘 다 내가 근데 나 아니 진짜 더 내일 그럼 이걸 니 좀 락 너가 이거 

Label : Topic2
Words : 태환 나 난 진짜 이렇게 근데 락원이 동현이 니 왜 아 락원 있음 락원이는 동현 야 하고 그리고 에어 에어가 

Label : Topic3
Words : 오늘 1 어떻게 내 니는 진짜 락원이 락원이는 왜 배드민턴 현직이 나 살짝 파이썬 니한테 다 아니고 머리 언제 아니 



In [6]:
model["김동현"]

[(0, 0.9930031666683617)]

In [7]:
# 사용자별로 토픽 분포 확인하기.
def show_authors(name):
    print("User :", name)
    print("Docs :", model.author2doc[name])
    print("\nTopic Distribution : ")
    pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [8]:
# "네오"의 토픽 분포 확인.
show_authors("김동현")

User : 김동현
Docs : [102, 104, 135, 136, 148, 150, 152, 205, 206, 211, 212, 216, 239, 401, 402, 405, 406, 427, 431, 468, 470, 475, 476, 477, 482, 487, 488, 489, 492, 495, 498, 548, 584, 585, 586, 668, 669, 693, 701, 934, 960, 964, 973, 974]

Topic Distribution : 
[('Topic0', 0.9930031666683617)]


In [9]:
[model[author] for author in model.id2author.values()]

[[(0, 0.9930031666683617)],
 [(1, 0.9961430204373166)],
 [(1, 0.9993083076621049)],
 [(2, 0.9983250949373436)],
 [(3, 0.990095725131695)],
 [(0, 0.9958453399102004)]]

In [10]:
[author for author in model.id2author.values()]

['김동현', '김락원 형 15', '김태환 형 17', '김표선 형 16', '이현직 16', '허정훈이 형 15']

In [11]:
# Hellinger Distance를 이용하여 비슷한 토픽을 가진 사용자를 추정하는 함수.


# author-topic 분포 만들기
author_vecs = [model[author] for author in model.id2author.values()]
 
def similarity(vec1, vec2):
    '''Get similarity between two vectors'''
    dist = hellinger(matutils.sparse2full(vec1, model.num_topics),
                     matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    
    return sim
 
def get_sims(vec):
    '''Get similarity of vector to all authors.'''
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims
 
def get_table(name, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author names, and author sizes.
    Return `top_n` authors as a dataframe.
    
    '''
    
    # 유사도 측정하기
    sims = get_sims(model[name])
 
    # 저자별 정보 정렬하기
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        
        table.append((author_name, sim, author_size))
    
    # 사용자 패턴 분석 결과를 Dataframe으로 만들기
    df = pd.DataFrame(table, columns=["Author", "Score", "Size"])
    df = df.sort_values("Score", ascending=False)[:top_n]
    
    
    return df

In [12]:
# 사용자별 대화 패턴 검증
get_table("김동현")

,Author,Score,Size
0,김동현,1.000000,44
5,허정훈이 형 15,0.998993,246
4,이현직 16,0.501061,63
1,김락원 형 15,0.500680,60
3,김표선 형 16,0.500543,250
2,김태환 형 17,0.500481,317


> '김동현'은 말을 44번 했고, 말을 246번 한 '허정훈이 형 15'와 가장 비슷